### Enviroment

In [ ]:
from tqdm import tqdm
import datetime as dt
from scipy.stats import uniform
from scipy.stats import truncnorm
from scipy.stats import bernoulli
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import mysql.connector
password_sql = 'YOUR_PASSWORD'

In [ ]:
mnq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "mnq")

### Data Cleaning

```sql
SHOW GLOBAL VARIABLES LIKE 'local_infile';

SET GLOBAL local_infile=1;

CREATE TABLE mnq_historical_data (event_ts BIGINT NOT NULL, rtype INT NOT NULL, publisher_id INT NOT NULL, instrument_id BIGINT NOT NULL, open DECIMAL(18,2), high DECIMAL(18,2), low DECIMAL(18,2), close DECIMAL(18,2), volume INT, symbol VARCHAR(20));


LOAD DATA LOCAL INFILE "C:/Users/nicho/Desktop/Nuova cartella/MNQ_historical_data.csv" INTO TABLE mnq_historical_data
FIELDS TERMINATED BY ","
LINES TERMINATED BY "\n"
IGNORE 1 LINES
(event_ts, rtype, publisher_id, instrument_id, open, high, low, close, volume, symbol);


CREATE TABLE mnq_historical_data1 
SELECT event_ts, open, high, low, close, volume, symbol 
FROM mnq_historical_data;


SET sql_safe_updates = 0;


ALTER TABLE mnq_historical_data1 ADD COLUMN date TIMESTAMP;


UPDATE mnq_historical_data1 
SET date = FROM_UNIXTIME(event_ts/1000000000); -- nanoseconds precision


ALTER TABLE mnq_historical_data1 DROP COLUMN event_ts;


ALTER TABLE mnq_historical_data1 ADD COLUMN date_ny DATETIME;


UPDATE mnq_historical_data1 
SET date_ny = CONVERT_TZ(date, 'UTC', 'America/New_York');


ALTER TABLE mnq_historical_data1 DROP COLUMN date;

-- Crea la tabella dei contratti front month (con maggiore volume giornaliero)
CREATE TABLE front_month
WITH daily_volume AS (
    SELECT DATE(date_ny) AS date, symbol, SUM(volume) AS total_volume
    FROM mnq_historical_data1
    GROUP BY DATE(date_ny), symbol
),
front_month AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY date ORDER BY total_volume DESC) AS rnk
    FROM daily_volume
)
SELECT date, symbol, total_volume  
FROM front_month 
WHERE rnk = 1 
ORDER BY date;

-- Conta quante volte cambia il front month per anno
WITH front_with_lag AS (
    SELECT date, symbol AS front_symbol, 
           LAG(symbol) OVER (ORDER BY date) AS prev_symbol,
           EXTRACT(YEAR FROM date) AS trade_year
    FROM front_month
)
SELECT trade_year, COUNT(*) AS switch_count
FROM front_with_lag
WHERE front_symbol != prev_symbol AND prev_symbol IS NOT NULL
GROUP BY trade_year
ORDER BY trade_year;

-- Filtra i dati storici per includere solo i contratti front month
CREATE TABLE mnq_historical_data2 
SELECT * 
FROM mnq_historical_data1 
WHERE (DATE(date_ny), symbol) IN (SELECT date, symbol FROM front_month);

-- Controlla che i contratti rimasti nel database siano in linea con quelli front month
WITH date_symbol AS (SELECT DISTINCT DATE(date_ny) AS date, symbol FROM mnq_historical_data2),
test AS (SELECT front_month.date, front_month.symbol AS front_month_symbol, date_symbol.symbol AS traded_symbol FROM front_month JOIN date_symbol ON front_month.date = date_symbol.date)
SELECT COUNT(*) AS contract_divergence  FROM test WHERE front_month_symbol != traded_symbol;

CREATE TABLE mnq_historical_data_5m (date_ny DATETIME NOT NULL, open decimal(18,2), high decimal(18,2), low decimal(18,2), close decimal(18,2));

LOAD DATA LOCAL INFILE "C:/Users/nicho/Desktop/Trading Space/nq_historical_data_5m.csv" INTO TABLE mnq_historical_data_5m
FIELDS TERMINATED BY ","
LINES TERMINATED BY "\n"
IGNORE 1 LINES
(date_ny, open, high, low, close);
```

### Data for Strategies

In [ ]:
mnq_historical_data = pd.read_sql('Select * from mnq_historical_data2', mnq_connection_sql) 
mnq_historical_data.sort_values(by=['date_ny']).reset_index(drop=True)

In [ ]:
mnq_historical_data = mnq_historical_data.set_index('date_ny') #perchè.agg può essere eseguita solo sugli indici
mnq_historical_data_5m = mnq_historical_data.resample('5min').agg({'open': 'first','high': 'max','low': 'min','close': 'last'}) #Dove non trova nulla riporta tutti 0
mnq_historical_data_5m = mnq_historical_data_5m.replace(0, np.nan).dropna(how = 'all') #sostituisco gli 0 con Nan e elimino tutte le righe che presentano almeno un nan

In [ ]:
mnq_historical_data_5m.to_csv('MNQ_historical_data_5m.csv', index = True)

```sql
CREATE TABLE nq_historical_data_5m (date_ny DATETIME NOT NULL, open decimal(18,2), high decimal(18,2), low decimal(18,2), close decimal(18,2));

LOAD DATA LOCAL INFILE "C:/Users/nicho/Desktop/Trading Space/Progetto QQQ/NQ_historical_data_5m.csv" INTO TABLE nq_historical_data_5m
FIELDS TERMINATED BY ","
LINES TERMINATED BY "\n"
IGNORE 1 LINES
(date_ny, open, high, low, close);

create table mnq_historical_data_5m_1 select * from mnq_historical_data_5m where time(date_ny) <= "15:55:00";

WITH
list_close as (select date(date_ny) as date, max(time(date_ny)) as close_time from mnq_historical_data_5m_1 group by date(date_ny)),
list_day_close as (select * from list_close where close_time != '15:55:00')
delete from mnq_historical_data_5m_1 where date(date_ny) in (select date from list_day_close);

WITH
list_close as (select date(date_ny) as date, max(time(date_ny)) as close_time from mnq_historical_data_5m_1 group by date(date_ny)),
list_day_close as (select * from list_close where close_time != '15:55:00')
select * from list_day_close
```

### MNQ Test 1 

In [ ]:
mnq_test1_db = pd.read_sql('Select * from mnq_historical_data_5m_1 where time(date_ny) >= "02:00:00" and date(date_ny) < "2025-11-01"', mnq_connection_sql)
print(mnq_test1_db.dtypes)

mnq_test1_db['date'] = mnq_test1_db['date_ny'].dt.date
mnq_test1_db['time'] = mnq_test1_db['date_ny'].dt.time

mnq_test1_db = mnq_test1_db.sort_values(by = ['date','time']).reset_index(drop=True)

In [ ]:
class trading_strategy_sim_slip:
    def __init__(self):
        # --- Settings ---
        self.initial_account = 25000
        self.fee = 0.25     #Per part (no roundturn)
        self.point_value = 2
        
        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.account = []

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = round(min((self.account[-1] * 0.01) / (self.risk*self.point_value),(4 * self.account[-1]) / (self.opening_price*self.point_value)),0)
        
        self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, step=10, slip=0.25):
        remaining = units
        level = 0
        total_cost = 0

        while remaining > 0:
            qty = min(step, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            total_cost += qty * exec_price
            remaining -= qty
            level += 1

        return total_cost / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy')
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell')
            self.direction_history.append('Sell')


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = ((self.real_sell_price - self.real_buy_price) * self.units * self.point_value - 2 * self.units * self.fee)

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)


    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Buy": self.real_buy_price_history,
            "Average_Sell": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account})  # skip initial capital

In [ ]:
mnq_test1 = trading_strategy_sim_slip()

In [ ]:
for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test1.starting_backtest(mnq_test1_db['date'].iloc[x])
        

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        #BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-1]) > 0:
            mnq_test1.buy_pos = 1
            mnq_test1.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test1.opening_price = mnq_test1.buy_price
            mnq_test1.stop_loss = mnq_test1_db['low'].iloc[x-1]
            mnq_test1.take_profit = (mnq_test1.opening_price - mnq_test1.stop_loss)*10 + mnq_test1.opening_price
            mnq_test1.risk = abs(mnq_test1.opening_price - mnq_test1.stop_loss)

            mnq_test1.apply_entry('Buy')

        #SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-1]) < 0:
            mnq_test1.sell_pos = 1
            mnq_test1.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test1.opening_price = mnq_test1.sell_price
            mnq_test1.stop_loss = mnq_test1_db['high'].iloc[x-1]
            mnq_test1.take_profit = (mnq_test1.opening_price - mnq_test1.stop_loss)*10 + mnq_test1.opening_price
            mnq_test1.risk = abs(mnq_test1.opening_price - mnq_test1.stop_loss)

            mnq_test1.apply_entry('Sell')

    # ================= LONG =================
    if mnq_test1.buy_pos == 1 and mnq_test1.sell_pos == 0:

        if mnq_test1_db['low'].iloc[x] <= mnq_test1.stop_loss:
            mnq_test1.sell_pos = 1
            mnq_test1.sell_price = mnq_test1.stop_loss

            mnq_test1.apply_stop('Buy')

        elif mnq_test1_db['high'].iloc[x] >= mnq_test1.take_profit:
            mnq_test1.sell_pos = 1
            mnq_test1.sell_price = mnq_test1.take_profit
            mnq_test1.real_sell_price = mnq_test1.sell_price  # NO slippage

    # ================= SHORT =================
    if mnq_test1.buy_pos == 0 and mnq_test1.sell_pos == 1:

        if mnq_test1_db['high'].iloc[x] >= mnq_test1.stop_loss:
            mnq_test1.buy_pos = 1
            mnq_test1.buy_price = mnq_test1.stop_loss

            mnq_test1.apply_stop('Sell')

        elif mnq_test1_db['low'].iloc[x] <= mnq_test1.take_profit:
            mnq_test1.buy_pos = 1
            mnq_test1.buy_price = mnq_test1.take_profit
            mnq_test1.real_buy_price = mnq_test1.buy_price  # NO slippage

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test1.buy_pos == 1 and mnq_test1.sell_pos == 0:
            mnq_test1.sell_pos = 1
            mnq_test1.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test1.real_sell_price = mnq_test1.sell_price

        if mnq_test1.buy_pos == 0 and mnq_test1.sell_pos == 1:
            mnq_test1.buy_pos = 1
            mnq_test1.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test1.real_buy_price = mnq_test1.buy_price

    # ================= RECORD =================
    if mnq_test1.buy_pos == 1 and mnq_test1.sell_pos == 1:
        mnq_test1.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test1.reset()


### MNQ Test 1 Simulation

In [ ]:
fig0 = go.Figure()
fig0.add_trace(go.Scatter(x = mnq_test1.dates, y = mnq_test1.account, mode = 'lines', name = 'MNQ Test 1'))
fig0.update_layout(title = dict(text = "MNQ Test 1 Simulation", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

In [ ]:
print("Average Profit per Unit:", np.mean(mnq_test1.unit_trade))

### MNQ Test 1 Traded Units

In [ ]:
fig0 = go.Figure()
fig0.add_trace(go.Scatter(x = mnq_test1.dates, y = mnq_test1.unit_list, mode = 'lines', name = 'Test 1'))
fig0.update_layout(title = dict(text = "MNQ Test 1 Traded Units", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### QQQ Slippage Results

In [ ]:
qqq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "qqq")

In [ ]:
test1_db = pd.read_sql('Select * from test1_data where date(date_ny) >= "2019-05-05"', qqq_connection_sql)
print(test1_db.dtypes)

test1_db['date'] = test1_db['date_ny'].dt.date
test1_db['time'] = test1_db['date_ny'].dt.time

test1_db = test1_db.sort_values(by=["date", "time"]).reset_index(drop=True)

In [ ]:
class trading_strategy_sim_slip_setup1:
    def __init__(self):
        # --- Account ---
        self.initial_account = 25000
        self.account = [self.initial_account]

        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real, with slippage) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.fee = 0.0005

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = int(
            min(
                (self.account[-1] * 0.01) / self.risk,
                (4 * self.account[-1]) / self.opening_price
            )
        )
        self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, step=750, slip=0.01):
        remaining = units
        level = 0
        total_cost = 0

        while remaining > 0:
            qty = min(step, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            total_cost += qty * exec_price
            remaining -= qty
            level += 1

        return total_cost / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy')
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell')
            self.direction_history.append('Sell')

    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = (
            (self.real_sell_price - self.real_buy_price) * self.units
            - 2 * self.units * self.fee
        )

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)

    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Entry": self.real_buy_price_history,
            "Average_Exit": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account})  # skip initial capital

In [ ]:
qqq_slip = trading_strategy_sim_slip_setup1()

In [ ]:
for x in tqdm(range(0, len(test1_db))):

    if x == 0:
        qqq_slip.starting_backtest(test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if test1_db['time'].iloc[x] == dt.time(9,35,0):

        if test1_db['direction'].iloc[x] == 'Buy':
            qqq_slip.buy_pos = 1
            qqq_slip.buy_price = test1_db['open'].iloc[x]
            qqq_slip.opening_price = qqq_slip.buy_price
            qqq_slip.stop_loss = test1_db['stop_loss'].iloc[x]
            qqq_slip.take_profit = test1_db['target'].iloc[x]
            qqq_slip.risk = abs(qqq_slip.opening_price - qqq_slip.stop_loss)

            qqq_slip.apply_entry('Buy')

        if test1_db['direction'].iloc[x] == 'Sell':
            qqq_slip.sell_pos = 1
            qqq_slip.sell_price = test1_db['open'].iloc[x]
            qqq_slip.opening_price = qqq_slip.sell_price
            qqq_slip.stop_loss = test1_db['stop_loss'].iloc[x]
            qqq_slip.take_profit = test1_db['target'].iloc[x]
            qqq_slip.risk = abs(qqq_slip.opening_price - qqq_slip.stop_loss)

            qqq_slip.apply_entry('Sell')

    # ================= LONG =================
    if qqq_slip.buy_pos == 1 and qqq_slip.sell_pos == 0:

        if test1_db['low'].iloc[x] <= qqq_slip.stop_loss:
            qqq_slip.sell_pos = 1
            qqq_slip.sell_price = qqq_slip.stop_loss

            qqq_slip.apply_stop('Buy')

        elif test1_db['high'].iloc[x] >= qqq_slip.take_profit:
            qqq_slip.sell_pos = 1
            qqq_slip.sell_price = qqq_slip.take_profit
            qqq_slip.real_sell_price = qqq_slip.sell_price  # NO slippage

    # ================= SHORT =================
    if qqq_slip.buy_pos == 0 and qqq_slip.sell_pos == 1:

        if test1_db['high'].iloc[x] >= qqq_slip.stop_loss:
            qqq_slip.buy_pos = 1
            qqq_slip.buy_price = qqq_slip.stop_loss

            qqq_slip.apply_stop('Sell')

        elif test1_db['low'].iloc[x] <= qqq_slip.take_profit:
            qqq_slip.buy_pos = 1
            qqq_slip.buy_price = qqq_slip.take_profit
            qqq_slip.real_buy_price = qqq_slip.buy_price  # NO slippage

    # ================= END DAY =================
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if qqq_slip.buy_pos == 1 and qqq_slip.sell_pos == 0:
            qqq_slip.sell_pos = 1
            qqq_slip.sell_price = test1_db['close'].iloc[x]
            qqq_slip.real_sell_price = qqq_slip.sell_price

        if qqq_slip.buy_pos == 0 and qqq_slip.sell_pos == 1:
            qqq_slip.buy_pos = 1
            qqq_slip.buy_price = test1_db['close'].iloc[x]
            qqq_slip.real_buy_price = qqq_slip.buy_price

    # ================= RECORD =================
    if qqq_slip.buy_pos == 1 and qqq_slip.sell_pos == 1:
        qqq_slip.record_trade(test1_db['date'].iloc[x])
        qqq_slip.reset()


### MNQ Test 1 Slip. VS QQQ Slip.

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test1.dates, y = mnq_test1.account, mode = 'lines', name = 'MNQ Test 1'))
fig1.add_trace(go.Scatter(x = qqq_slip.dates, y = qqq_slip.account, mode = 'lines', name = 'QQQ Slip'))
fig1.update_layout(title = dict(text = "MNQ Test 1 Slip. VS QQQ Slip. Simulation", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### MNQ Test 1 Slip. VS QQQ Slip. PnL

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test1.dates, y = mnq_test1.trades_pnl, mode = 'lines', name = 'MNQ Test 1'))
fig1.add_trace(go.Scatter(x = qqq_slip.dates, y = qqq_slip.trades_pnl, mode = 'lines', name = 'QQQ Slip'))
fig1.update_layout(title = dict(text = "MNQ Test 1 Slip. VS QQQ Slip. PnL", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### MNQ Test 2 A

In [ ]:
mnq_test2A = trading_strategy_sim_slip()

In [ ]:
for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test2A.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test2A.buy_pos = 1
            mnq_test2A.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test2A.opening_price = mnq_test2A.buy_price
            mnq_test2A.stop_loss = mnq_test1_db['low'].iloc[x-1]
            mnq_test2A.take_profit = (mnq_test2A.opening_price - mnq_test2A.stop_loss) * 10 + mnq_test2A.opening_price
            mnq_test2A.risk = abs(mnq_test2A.opening_price - mnq_test2A.stop_loss)

            mnq_test2A.apply_entry('Buy')

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test2A.sell_pos = 1
            mnq_test2A.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test2A.opening_price = mnq_test2A.sell_price
            mnq_test2A.stop_loss = mnq_test1_db['high'].iloc[x-1]
            mnq_test2A.take_profit = (mnq_test2A.opening_price - mnq_test2A.stop_loss) * 10 + mnq_test2A.opening_price
            mnq_test2A.risk = abs(mnq_test2A.opening_price - mnq_test2A.stop_loss)

            mnq_test2A.apply_entry('Sell')

    # ================= LONG =================
    if mnq_test2A.buy_pos == 1 and mnq_test2A.sell_pos == 0:

        if mnq_test1_db['low'].iloc[x] <= mnq_test2A.stop_loss:
            mnq_test2A.sell_pos = 1
            mnq_test2A.sell_price = mnq_test2A.stop_loss

            mnq_test2A.apply_stop('Buy')

        elif mnq_test1_db['high'].iloc[x] >= mnq_test2A.take_profit:
            mnq_test2A.sell_pos = 1
            mnq_test2A.sell_price = mnq_test2A.take_profit
            mnq_test2A.real_sell_price = mnq_test2A.sell_price  # NO slippage

    # ================= SHORT =================
    if mnq_test2A.buy_pos == 0 and mnq_test2A.sell_pos == 1:

        if mnq_test1_db['high'].iloc[x] >= mnq_test2A.stop_loss:
            mnq_test2A.buy_pos = 1
            mnq_test2A.buy_price = mnq_test2A.stop_loss

            mnq_test2A.apply_stop('Sell')

        elif mnq_test1_db['low'].iloc[x] <= mnq_test2A.take_profit:
            mnq_test2A.buy_pos = 1
            mnq_test2A.buy_price = mnq_test2A.take_profit
            mnq_test2A.real_buy_price = mnq_test2A.buy_price  # NO slippage

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test2A.buy_pos == 1 and mnq_test2A.sell_pos == 0:
            mnq_test2A.sell_pos = 1
            mnq_test2A.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test2A.real_sell_price = mnq_test2A.sell_price

        if mnq_test2A.buy_pos == 0 and mnq_test2A.sell_pos == 1:
            mnq_test2A.buy_pos = 1
            mnq_test2A.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test2A.real_buy_price = mnq_test2A.buy_price

    # ================= RECORD =================
    if mnq_test2A.buy_pos == 1 and mnq_test2A.sell_pos == 1:
        mnq_test2A.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test2A.reset()


### MNQ Test 2 B

In [ ]:
mnq_test2B = trading_strategy_sim_slip()

In [ ]:
for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test2B.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-1]) > 0 and (mnq_test1_db['close'].iloc[x-2] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test2B.buy_pos = 1
            mnq_test2B.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test2B.opening_price = mnq_test2B.buy_price
            mnq_test2B.stop_loss = mnq_test1_db['low'].iloc[x-1]
            mnq_test2B.take_profit = (mnq_test2B.opening_price - mnq_test2B.stop_loss) * 10 + mnq_test2B.opening_price
            mnq_test2B.risk = abs(mnq_test2B.opening_price - mnq_test2B.stop_loss)

            mnq_test2B.apply_entry('Buy')

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-1]) < 0 and (mnq_test1_db['close'].iloc[x-2] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test2B.sell_pos = 1
            mnq_test2B.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test2B.opening_price = mnq_test2B.sell_price
            mnq_test2B.stop_loss = mnq_test1_db['high'].iloc[x-1]
            mnq_test2B.take_profit = (mnq_test2B.opening_price - mnq_test2B.stop_loss) * 10 + mnq_test2B.opening_price
            mnq_test2B.risk = abs(mnq_test2B.opening_price - mnq_test2B.stop_loss)

            mnq_test2B.apply_entry('Sell')

    # ================= LONG =================
    if mnq_test2B.buy_pos == 1 and mnq_test2B.sell_pos == 0:

        if mnq_test1_db['low'].iloc[x] <= mnq_test2B.stop_loss:
            mnq_test2B.sell_pos = 1
            mnq_test2B.sell_price = mnq_test2B.stop_loss

            mnq_test2B.apply_stop('Buy')

        elif mnq_test1_db['high'].iloc[x] >= mnq_test2B.take_profit:
            mnq_test2B.sell_pos = 1
            mnq_test2B.sell_price = mnq_test2B.take_profit
            mnq_test2B.real_sell_price = mnq_test2B.sell_price  # NO slippage

    # ================= SHORT =================
    if mnq_test2B.buy_pos == 0 and mnq_test2B.sell_pos == 1:

        if mnq_test1_db['high'].iloc[x] >= mnq_test2B.stop_loss:
            mnq_test2B.buy_pos = 1
            mnq_test2B.buy_price = mnq_test2B.stop_loss

            mnq_test2B.apply_stop('Sell')

        elif mnq_test1_db['low'].iloc[x] <= mnq_test2B.take_profit:
            mnq_test2B.buy_pos = 1
            mnq_test2B.buy_price = mnq_test2B.take_profit
            mnq_test2B.real_buy_price = mnq_test2B.buy_price  # NO slippage

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test2B.buy_pos == 1 and mnq_test2B.sell_pos == 0:
            mnq_test2B.sell_pos = 1
            mnq_test2B.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test2B.real_sell_price = mnq_test2B.sell_price

        if mnq_test2B.buy_pos == 0 and mnq_test2B.sell_pos == 1:
            mnq_test2B.buy_pos = 1
            mnq_test2B.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test2B.real_buy_price = mnq_test2B.buy_price

    # ================= RECORD =================
    if mnq_test2B.buy_pos == 1 and mnq_test2B.sell_pos == 1:
        mnq_test2B.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test2B.reset()


### MNQ Test 1 Slip. VS MNQ Test 2A Slip. VS MNQ Test 2B Slip.

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test1.dates, y = mnq_test1.account, mode = 'lines', name = 'MNQ Test 1'))
fig1.add_trace(go.Scatter(x = mnq_test2A.dates, y = mnq_test2A.account, mode = 'lines', name = 'MNQ Test 2A'))
fig1.add_trace(go.Scatter(x = mnq_test2B.dates, y = mnq_test2B.account, mode = 'lines', name = 'MNQ Test 2B'))
fig1.update_layout(title = dict(text = "MNQ Test 1 Slip. VS MNQ Test 2A Slip. VS MNQ Test 2B Slip.", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

In [ ]:
print('MNQ Test 1',
      'Account:', mnq_test1.account[-1], 
      'Average Profit per Unit:', np.mean(mnq_test1.unit_trade), 
      'Number of Trades:', len(mnq_test1.trades_pnl))
print('MNQ Test 2A',
      'Account:', mnq_test2A.account[-1],
      'Average Profit per Unit:', np.mean(mnq_test2A.unit_trade),
      'Number of Trades:', len(mnq_test2A.trades_pnl))

print('MNQ Test 2B',
      'Account:', mnq_test2B.account[-1],
      'Average Profit per Unit:', np.mean(mnq_test2B.unit_trade),
      'Number of Trades:', len(mnq_test2B.trades_pnl))

### MNQ Test 3

In [ ]:
mnq_test3 = trading_strategy_sim_slip()

In [ ]:
for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test3.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test3.buy_pos = 1
            mnq_test3.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test3.opening_price = mnq_test3.buy_price
            mnq_test3.stop_loss = min(mnq_test1_db['low'].iloc[x-1], mnq_test1_db['low'].iloc[x-2])
            mnq_test3.take_profit = (mnq_test3.opening_price - mnq_test3.stop_loss) * 10 + mnq_test3.opening_price
            mnq_test3.risk = abs(mnq_test3.opening_price - mnq_test3.stop_loss)

            mnq_test3.apply_entry('Buy')

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test3.sell_pos = 1
            mnq_test3.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test3.opening_price = mnq_test3.sell_price
            mnq_test3.stop_loss = max(mnq_test1_db['high'].iloc[x-1], mnq_test1_db['high'].iloc[x-2])
            mnq_test3.take_profit = (mnq_test3.opening_price - mnq_test3.stop_loss) * 10 + mnq_test3.opening_price
            mnq_test3.risk = abs(mnq_test3.opening_price - mnq_test3.stop_loss)

            mnq_test3.apply_entry('Sell')

    # ================= LONG =================
    if mnq_test3.buy_pos == 1 and mnq_test3.sell_pos == 0:

        if mnq_test1_db['low'].iloc[x] <= mnq_test3.stop_loss:
            mnq_test3.sell_pos = 1
            mnq_test3.sell_price = mnq_test3.stop_loss

            mnq_test3.apply_stop('Buy')

        elif mnq_test1_db['high'].iloc[x] >= mnq_test3.take_profit:
            mnq_test3.sell_pos = 1
            mnq_test3.sell_price = mnq_test3.take_profit
            mnq_test3.real_sell_price = mnq_test3.sell_price  # NO slippage

    # ================= SHORT =================
    if mnq_test3.buy_pos == 0 and mnq_test3.sell_pos == 1:

        if mnq_test1_db['high'].iloc[x] >= mnq_test3.stop_loss:
            mnq_test3.buy_pos = 1
            mnq_test3.buy_price = mnq_test3.stop_loss

            mnq_test3.apply_stop('Sell')

        elif mnq_test1_db['low'].iloc[x] <= mnq_test3.take_profit:
            mnq_test3.buy_pos = 1
            mnq_test3.buy_price = mnq_test3.take_profit
            mnq_test3.real_buy_price = mnq_test3.buy_price  # NO slippage

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test3.buy_pos == 1 and mnq_test3.sell_pos == 0:
            mnq_test3.sell_pos = 1
            mnq_test3.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test3.real_sell_price = mnq_test3.sell_price

        if mnq_test3.buy_pos == 0 and mnq_test3.sell_pos == 1:
            mnq_test3.buy_pos = 1
            mnq_test3.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test3.real_buy_price = mnq_test3.buy_price

    # ================= RECORD =================
    if mnq_test3.buy_pos == 1 and mnq_test3.sell_pos == 1:
        mnq_test3.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test3.reset()


### MNQ Test 3 Slip. VS MNQ Test 2A Slip.

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test3.dates, y = mnq_test3.account, mode = 'lines', name = 'MNQ Test 3'))
fig1.add_trace(go.Scatter(x = mnq_test2A.dates, y = mnq_test2A.account, mode = 'lines', name = 'MNQ Test 2A'))
fig1.update_layout(title = dict(text = "MNQ Test 3 Slip. VS MNQ Test 2A Slip.", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

In [ ]:
print('MNQ Test 3',
      'Account:', mnq_test3.account[-1], 
      'Average Profit per Unit:', np.mean(mnq_test3.unit_trade), 
      'Number of Trades:', len(mnq_test3.trades_pnl))
print('MNQ Test 2A',
      'Account:', mnq_test2A.account[-1],
      'Average Profit per Unit:', np.mean(mnq_test2A.unit_trade),
      'Number of Trades:', len(mnq_test2A.trades_pnl))

### MNQ Test 3 Slip. Traded Unites 

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test3.dates, y = mnq_test3.unit_list, mode = 'lines', name = 'MNQ Test 3'))
fig1.update_layout(title = dict(text = "MNQ Test 3 Slip. Traded Unites", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### Test 4

In [ ]:
mnq_test4 = trading_strategy_sim_slip()

In [ ]:
for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test4.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test4.buy_pos = 1
            mnq_test4.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test4.opening_price = mnq_test4.buy_price
            mnq_test4.stop_loss = min(mnq_test1_db['low'].iloc[x-1], mnq_test1_db['low'].iloc[x-2])
            mnq_test4.take_profit = (mnq_test4.opening_price - mnq_test4.stop_loss) * 10 + mnq_test4.opening_price
            mnq_test4.risk = abs(mnq_test4.opening_price - mnq_test4.stop_loss)
            mnq_test4.stop_to_be = False  

            mnq_test4.apply_entry('Buy')

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test4.sell_pos = 1
            mnq_test4.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test4.opening_price = mnq_test4.sell_price
            mnq_test4.stop_loss = max(mnq_test1_db['high'].iloc[x-1], mnq_test1_db['high'].iloc[x-2])
            mnq_test4.take_profit = (mnq_test4.opening_price - mnq_test4.stop_loss) * 10 + mnq_test4.opening_price
            mnq_test4.risk = abs(mnq_test4.opening_price - mnq_test4.stop_loss)
            mnq_test4.stop_to_be = False  

            mnq_test4.apply_entry('Sell')

    # ================= LONG =================
    if mnq_test4.buy_pos == 1 and mnq_test4.sell_pos == 0:

        # Stop loss Iniziale
        if mnq_test1_db['low'].iloc[x] <= mnq_test4.stop_loss:
            mnq_test4.sell_pos = 1
            mnq_test4.sell_price = mnq_test4.stop_loss
            mnq_test4.apply_stop('Buy')

        # Take profit normale
        elif mnq_test1_db['high'].iloc[x] >= mnq_test4.take_profit:
            mnq_test4.sell_pos = 1
            mnq_test4.sell_price = mnq_test4.take_profit
            mnq_test4.real_sell_price = mnq_test4.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif not mnq_test4.stop_to_be:
            unrealized_profit = mnq_test1_db['close'].iloc[x] - mnq_test4.real_buy_price
            real_risk = abs(mnq_test4.real_buy_price - mnq_test4.stop_loss)
            if unrealized_profit >= 1 * real_risk:
                mnq_test4.stop_loss = mnq_test4.real_buy_price  # stop a BE
                mnq_test4.stop_to_be = True

    # ================= SHORT =================
    if mnq_test4.buy_pos == 0 and mnq_test4.sell_pos == 1:

        # Stop loss iniziale
        if mnq_test1_db['high'].iloc[x] >= mnq_test4.stop_loss:
            mnq_test4.buy_pos = 1
            mnq_test4.buy_price = mnq_test4.stop_loss
            mnq_test4.apply_stop('Sell')

        # Take profit normale
        elif mnq_test1_db['low'].iloc[x] <= mnq_test4.take_profit:
            mnq_test4.buy_pos = 1
            mnq_test4.buy_price = mnq_test4.take_profit
            mnq_test4.real_buy_price = mnq_test4.buy_price  # NO slippage

    # ================= STOP MANAGEMENT =================
        elif not mnq_test4.stop_to_be:
            unrealized_profit = mnq_test4.real_sell_price - mnq_test1_db['close'].iloc[x]
            real_risk = abs(mnq_test4.real_sell_price - mnq_test4.stop_loss)
            if unrealized_profit >= 1 * real_risk:
                mnq_test4.stop_loss = mnq_test4.real_sell_price  # stop a BE
                mnq_test4.stop_to_be = True

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test4.buy_pos == 1 and mnq_test4.sell_pos == 0:
            mnq_test4.sell_pos = 1
            mnq_test4.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test4.real_sell_price = mnq_test4.sell_price

        if mnq_test4.buy_pos == 0 and mnq_test4.sell_pos == 1:
            mnq_test4.buy_pos = 1
            mnq_test4.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test4.real_buy_price = mnq_test4.buy_price

    # ================= RECORD =================
    if mnq_test4.buy_pos == 1 and mnq_test4.sell_pos == 1:
        mnq_test4.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test4.reset()


### MNQ Test 3 Slip. VS MNQ Test 4 Slip.

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test3.dates, y = mnq_test3.account, mode = 'lines', name = 'MNQ Test 3'))
fig1.add_trace(go.Scatter(x = mnq_test4.dates, y = mnq_test4.account, mode = 'lines', name = 'MNQ Test 4'))
fig1.update_layout(title = dict(text = "MNQ Test 3 Slip. VS MNQ Test 4 Slip.", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### Define Drawdown Function

In [ ]:
def drawdowns_stats(equity_curve):

    # Calcolo drawdown punto per punto
    high_watermark = equity_curve.cummax()
    drawdown_value = equity_curve / high_watermark - 1

    drawdowns = []
    durations = []
    current_dd = []

    # Identificazione dei drawdown
    for dd in drawdown_value:
        if dd < 0:
            current_dd.append(dd)
        elif current_dd:  # if current_dd --> if len(current_dd) > 0
            drawdowns.append(min(current_dd))
            durations.append(len(current_dd))
            current_dd = []

    # Se la serie termina in drawdown
    if current_dd:
        drawdowns.append(min(current_dd))
        durations.append(len(current_dd))

    # Metriche 
    max_drawdown = min(drawdowns)
    average_drawdown = sum(drawdowns) / len(drawdowns)
    average_duration = sum(durations) / len(durations)
    tuw = (drawdown_value < 0).sum() / len(drawdown_value) if len(drawdown_value) > 0 else 0 # % di periodi in drowdown

    return {
        'Max Drawdown': max_drawdown,
        'Average Drawdown': average_drawdown,
        'Average DD Duration': average_duration,
        'Time Under Water': tuw
    }

### MNQ Test 3 Slip. VS MNQ Test 4 Slip. Strategy Comparison

In [ ]:
#Regression
x3 = np.arange(len(mnq_test3.dates)) #assegna un numero alle date [0,1,2 ...]
x4 = np.arange(len(mnq_test4.dates))

beta3, alpha3 = np.polyfit(x3, np.log(mnq_test3.account), 1) 
beta4, alpha4 = np.polyfit(x4, np.log(mnq_test4.account), 1)

# La funzione np.polyfit stima i coefficienti di un polinomio di grado x, nel mio caso ho impostato 1 perciò una retta (Y = a + bx). I coefficienti determinati sono quelli che minimizzano
# la somma degli errori quadratici. Nel mio caso: sommatoria((Yi - (a + bx)^2). 
# IMPORTANTE: la funzione riporta i risultati in ordine decrescente di grado. Quindi nel mio caso prima Beta e poi Alpha. Se ho una paroabola prima X^2 poi x poi c

linear_reg_mnq_test3 = alpha3 + beta3 * x3
linear_reg_mnq_test4 = alpha4 + beta4 * x4

fig5 = go.Figure()
fig5.add_trace(go.Scatter(x = mnq_test3.dates, y = np.log(mnq_test3.account), mode = 'lines', name = 'MNQ Test 3'))
fig5.add_trace(go.Scatter(x = mnq_test4.dates, y = np.log(mnq_test4.account), mode = 'lines', name = 'MNQ Test 4'))
fig5.update_layout(title = dict(text = "MNQ Test 3 Slip. VS MNQ Test 4 Slip. Strategy Comparison", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))


fig5.add_trace(go.Scatter(
    x = mnq_test3.dates,
    y = linear_reg_mnq_test3,
    mode = 'lines',
    name = 'Regression MNQ Test 3',
    line = dict(width = 3, color = 'Green')
   
))

fig5.add_trace(go.Scatter(
    x = mnq_test4.dates,
    y = linear_reg_mnq_test4,
    mode = 'lines',
    name = 'Regression MNQ Test 4',
    line = dict(width = 3, color = 'White')
))


In [ ]:
df_test3 = mnq_test3.trade_log()
equity_curve_test3 = df_test3['Account']
print(drawdowns_stats(equity_curve_test3))

df_test4 = mnq_test4.trade_log()
equity_curve_test4 = df_test4['Account']
print(drawdowns_stats(equity_curve_test4))

In [ ]:
#RMSE
from sklearn.metrics import mean_squared_error
mse_mnq_test3 = mean_squared_error(np.log(mnq_test3.account), linear_reg_mnq_test3)
mse_mnq_test4 = mean_squared_error(np.log(mnq_test4.account), linear_reg_mnq_test4)
print('Root Mean Squarred Error mnq_test3:', np.sqrt(mse_mnq_test3))
print('Root Mean Squarred Error mnq_test4:', np.sqrt(mse_mnq_test4))

### Test 4B

In [ ]:
mnq_test4B = trading_strategy_sim_slip()

In [ ]:
for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test4B.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test4B.buy_pos = 1
            mnq_test4B.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test4B.opening_price = mnq_test4B.buy_price
            mnq_test4B.stop_loss = min(mnq_test1_db['low'].iloc[x-1], mnq_test1_db['low'].iloc[x-2])
            mnq_test4B.take_profit = (mnq_test4B.opening_price - mnq_test4B.stop_loss) * 10 + mnq_test4B.opening_price
            mnq_test4B.risk = abs(mnq_test4B.opening_price - mnq_test4B.stop_loss)
            mnq_test4B.stop_to_be = False  

            mnq_test4B.apply_entry('Buy')

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test4B.sell_pos = 1
            mnq_test4B.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test4B.opening_price = mnq_test4B.sell_price
            mnq_test4B.stop_loss = max(mnq_test1_db['high'].iloc[x-1], mnq_test1_db['high'].iloc[x-2])
            mnq_test4B.take_profit = (mnq_test4B.opening_price - mnq_test4B.stop_loss) * 10 + mnq_test4B.opening_price
            mnq_test4B.risk = abs(mnq_test4B.opening_price - mnq_test4B.stop_loss)
            mnq_test4B.stop_to_be = False  

            mnq_test4B.apply_entry('Sell')

    # ================= LONG =================
    if mnq_test4B.buy_pos == 1 and mnq_test4B.sell_pos == 0:

        # Stop loss Iniziale
        if mnq_test1_db['low'].iloc[x] <= mnq_test4B.stop_loss:
            mnq_test4B.sell_pos = 1
            mnq_test4B.sell_price = mnq_test4B.stop_loss
            mnq_test4B.apply_stop('Buy')

        # Take profit normale
        elif mnq_test1_db['high'].iloc[x] >= mnq_test4B.take_profit:
            mnq_test4B.sell_pos = 1
            mnq_test4B.sell_price = mnq_test4B.take_profit
            mnq_test4B.real_sell_price = mnq_test4B.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif not mnq_test4B.stop_to_be:
            unrealized_profit = mnq_test1_db['close'].iloc[x] - mnq_test4B.real_buy_price
            real_risk = abs(mnq_test4B.real_buy_price - mnq_test4B.stop_loss)
            if unrealized_profit >= 2 * real_risk:
                mnq_test4B.stop_loss = mnq_test4B.real_buy_price  # stop a BE
                mnq_test4B.stop_to_be = True

    # ================= SHORT =================
    if mnq_test4B.buy_pos == 0 and mnq_test4B.sell_pos == 1:

        # Stop loss iniziale
        if mnq_test1_db['high'].iloc[x] >= mnq_test4B.stop_loss:
            mnq_test4B.buy_pos = 1
            mnq_test4B.buy_price = mnq_test4B.stop_loss
            mnq_test4B.apply_stop('Sell')

        # Take profit normale
        elif mnq_test1_db['low'].iloc[x] <= mnq_test4B.take_profit:
            mnq_test4B.buy_pos = 1
            mnq_test4B.buy_price = mnq_test4B.take_profit
            mnq_test4B.real_buy_price = mnq_test4B.buy_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif not mnq_test4B.stop_to_be:
            unrealized_profit = mnq_test4B.real_sell_price - mnq_test1_db['close'].iloc[x]
            real_risk = abs(mnq_test4B.real_sell_price - mnq_test4B.stop_loss)
            if unrealized_profit >= 2 * real_risk:
                mnq_test4B.stop_loss = mnq_test4B.real_sell_price  # stop a BE
                mnq_test4B.stop_to_be = True

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test4B.buy_pos == 1 and mnq_test4B.sell_pos == 0:
            mnq_test4B.sell_pos = 1
            mnq_test4B.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test4B.real_sell_price = mnq_test4B.sell_price

        if mnq_test4B.buy_pos == 0 and mnq_test4B.sell_pos == 1:
            mnq_test4B.buy_pos = 1
            mnq_test4B.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test4B.real_buy_price = mnq_test4B.buy_price

    # ================= RECORD =================
    if mnq_test4B.buy_pos == 1 and mnq_test4B.sell_pos == 1:
        mnq_test4B.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test4B.reset()


### MNQ Test 4 Slip. VS MNQ Test 4B Slip.

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test4.dates, y = mnq_test4.account, mode = 'lines', name = 'MNQ Test 4'))
fig1.add_trace(go.Scatter(x = mnq_test4B.dates, y = mnq_test4B.account, mode = 'lines', name = 'MNQ Test 4B'))
fig1.update_layout(title = dict(text = "MNQ Test 4 Slip. VS MNQ Test 4B Slip.", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### MNQ Test 4 Slip. VS MNQ Test 4B Slip.  Strategy Comparison

In [ ]:
#Regression
x4 = np.arange(len(mnq_test4.dates))
x4B = np.arange(len(mnq_test4B.dates)) 

beta4, alpha4 = np.polyfit(x4, np.log(mnq_test4.account), 1)
beta4B, alpha4B = np.polyfit(x4B, np.log(mnq_test4B.account), 1) 

linear_reg_mnq_test4 = alpha4 + beta4 * x4
linear_reg_mnq_test4B = alpha4B + beta4B * x4B

fig5 = go.Figure()
fig5.add_trace(go.Scatter(x = mnq_test4.dates, y = np.log(mnq_test4.account), mode = 'lines', name = 'MNQ Test 4'))
fig5.add_trace(go.Scatter(x = mnq_test4B.dates, y = np.log(mnq_test4B.account), mode = 'lines', name = 'MNQ Test 4B'))
fig5.update_layout(title = dict(text = "MNQ Test 4 Slip. VS MNQ Test 4B Slip.  Strategy Comparison", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

fig5.add_trace(go.Scatter(
    x = mnq_test4.dates,
    y = linear_reg_mnq_test4,
    mode = 'lines',
    name = 'Regression MNQ Test 4',
    line = dict(width = 3, color = 'White')
))

fig5.add_trace(go.Scatter(
    x = mnq_test4B.dates,
    y = linear_reg_mnq_test4B,
    mode = 'lines',
    name = 'Regression MNQ Test 4B',
    line = dict(width = 3, color = 'Green')   
))



In [ ]:
print(drawdowns_stats(equity_curve_test4))

df_test4B = mnq_test4B.trade_log()
equity_curve_test4B = df_test4B['Account']
print(drawdowns_stats(equity_curve_test4B))

In [ ]:
#RMSE
from sklearn.metrics import mean_squared_error
mse_mnq_test4 = mean_squared_error(np.log(mnq_test4.account), linear_reg_mnq_test4)
mse_mnq_test4B = mean_squared_error(np.log(mnq_test4B.account), linear_reg_mnq_test4B)
print('Root Mean Squarred Error mnq_test4:', np.sqrt(mse_mnq_test4))
print('Root Mean Squarred Error mnq_test4B:', np.sqrt(mse_mnq_test4B))

In [ ]:
print('Performance difference (%):', np.mean(np.log(mnq_test4.account)-np.log(mnq_test4B.account)))

### Test 4C

In [ ]:
mnq_test4C = trading_strategy_sim_slip()

for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test4C.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test4C.buy_pos = 1
            mnq_test4C.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test4C.opening_price = mnq_test4C.buy_price
            mnq_test4C.stop_loss = min(mnq_test1_db['low'].iloc[x-1], mnq_test1_db['low'].iloc[x-2])
            mnq_test4C.take_profit = (mnq_test4C.opening_price - mnq_test4C.stop_loss) * 10 + mnq_test4C.opening_price
            mnq_test4C.risk = abs(mnq_test4C.opening_price - mnq_test4C.stop_loss)
            mnq_test4C.stop_to_be = False
            mnq_test4C.stop_step_1 = False

            mnq_test4C.apply_entry('Buy')
            real_risk = abs(mnq_test4C.real_buy_price - mnq_test4C.stop_loss)

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test4C.sell_pos = 1
            mnq_test4C.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test4C.opening_price = mnq_test4C.sell_price
            mnq_test4C.stop_loss = max(mnq_test1_db['high'].iloc[x-1], mnq_test1_db['high'].iloc[x-2])
            mnq_test4C.take_profit = (mnq_test4C.opening_price - mnq_test4C.stop_loss) * 10 + mnq_test4C.opening_price
            mnq_test4C.risk = abs(mnq_test4C.opening_price - mnq_test4C.stop_loss)
            mnq_test4C.stop_to_be = False
            mnq_test4C.stop_step_1 = False
    
            mnq_test4C.apply_entry('Sell')
            real_risk = abs(mnq_test4C.real_sell_price - mnq_test4C.stop_loss)

    

    # ================= LONG =================
    if mnq_test4C.buy_pos == 1 and mnq_test4C.sell_pos == 0:

        unrealized_profit = mnq_test1_db['close'].iloc[x] - mnq_test4C.real_buy_price

        # Stop loss Iniziale
        if mnq_test1_db['low'].iloc[x] <= mnq_test4C.stop_loss:
            mnq_test4C.sell_pos = 1
            mnq_test4C.sell_price = mnq_test4C.stop_loss
            mnq_test4C.apply_stop('Buy')

        # Take profit normale
        elif mnq_test1_db['high'].iloc[x] >= mnq_test4C.take_profit:
            mnq_test4C.sell_pos = 1
            mnq_test4C.sell_price = mnq_test4C.take_profit
            mnq_test4C.real_sell_price = mnq_test4C.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif (not mnq_test4C.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            mnq_test4C.stop_loss = mnq_test4C.real_buy_price + 5 * real_risk
            mnq_test4C.stop_step_1 = True
            mnq_test4C.stop_to_be = True
            
        elif (not mnq_test4C.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            mnq_test4C.stop_loss = mnq_test4C.real_buy_price  # stop a BE
            mnq_test4C.stop_to_be = True
            
    # ================= SHORT =================
    if mnq_test4C.buy_pos == 0 and mnq_test4C.sell_pos == 1:

        unrealized_profit = mnq_test4C.real_sell_price - mnq_test1_db['close'].iloc[x]

        # Stop loss iniziale
        if mnq_test1_db['high'].iloc[x] >= mnq_test4C.stop_loss:
            mnq_test4C.buy_pos = 1
            mnq_test4C.buy_price = mnq_test4C.stop_loss
            mnq_test4C.apply_stop('Sell')

        # Take profit normale
        elif mnq_test1_db['low'].iloc[x] <= mnq_test4C.take_profit:
            mnq_test4C.buy_pos = 1
            mnq_test4C.buy_price = mnq_test4C.take_profit
            mnq_test4C.real_buy_price = mnq_test4C.buy_price  # NO slippage

        # ================= STOP MANAGEMENT ================
        elif (not mnq_test4C.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            mnq_test4C.stop_loss = mnq_test4C.real_sell_price - 5 * real_risk
            mnq_test4C.stop_step_1 = True
            mnq_test4C.stop_to_be = True
            
        elif (not mnq_test4C.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            mnq_test4C.stop_loss = mnq_test4C.real_sell_price  # stop a BE
            mnq_test4C.stop_to_be = True
            
    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test4C.buy_pos == 1 and mnq_test4C.sell_pos == 0:
            mnq_test4C.sell_pos = 1
            mnq_test4C.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test4C.real_sell_price = mnq_test4C.sell_price

        if mnq_test4C.buy_pos == 0 and mnq_test4C.sell_pos == 1:
            mnq_test4C.buy_pos = 1
            mnq_test4C.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test4C.real_buy_price = mnq_test4C.buy_price

    # ================= RECORD =================
    if mnq_test4C.buy_pos == 1 and mnq_test4C.sell_pos == 1:
        mnq_test4C.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test4C.reset()


### MNQ Test 4B Slip. VS MNQ Test 4C Slip.

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = mnq_test4.dates, y = mnq_test4.account, mode = 'lines', name = 'MNQ Test 4B'))
fig1.add_trace(go.Scatter(x = mnq_test4C.dates, y = mnq_test4C.account, mode = 'lines', name = 'MNQ Test 4C'))
fig1.update_layout(title = dict(text = "MNQ Test 4B Slip. VS MNQ Test 4C Slip.", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### MNQ Test 4B Slip. VS MNQ Test 4C Slip. Strategy Comparison

In [ ]:
#Regression
x4 = np.arange(len(mnq_test4.dates))
x4C = np.arange(len(mnq_test4C.dates)) 

beta4, alpha4 = np.polyfit(x4, np.log(mnq_test4.account), 1)
beta4C, alpha4C = np.polyfit(x4C, np.log(mnq_test4C.account), 1) 

linear_reg_mnq_test4 = alpha4 + beta4 * x4
linear_reg_mnq_test4C = alpha4C + beta4C * x4C

fig5 = go.Figure()
fig5.add_trace(go.Scatter(x = mnq_test4.dates, y = np.log(mnq_test4.account), mode = 'lines', name = 'MNQ Test 4'))
fig5.add_trace(go.Scatter(x = mnq_test4C.dates, y = np.log(mnq_test4C.account), mode = 'lines', name = 'MNQ Test 4C'))
fig5.update_layout(title = dict(text = "MNQ Test 4B Slip. VS MNQ Test 4C Slip. Strategy Comparison", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

fig5.add_trace(go.Scatter(
    x = mnq_test4.dates,
    y = linear_reg_mnq_test4,
    mode = 'lines',
    name = 'Regression MNQ Test 4',
    line = dict(width = 3, color = 'White')
))

fig5.add_trace(go.Scatter(
    x = mnq_test4C.dates,
    y = linear_reg_mnq_test4C,
    mode = 'lines',
    name = 'Regression MNQ Test 4C',
    line = dict(width = 3, color = 'Green')   
))



In [ ]:
print(drawdowns_stats(equity_curve_test4))

df_test4C = mnq_test4C.trade_log()
equity_curve_test4C = df_test4C['Account']
print(drawdowns_stats(equity_curve_test4C))

In [ ]:
#RMSE
from sklearn.metrics import mean_squared_error
mse_mnq_test4 = mean_squared_error(np.log(mnq_test4.account), linear_reg_mnq_test4)
mse_mnq_test4C = mean_squared_error(np.log(mnq_test4C.account), linear_reg_mnq_test4C)
print('Root Mean Squarred Error mnq_test4:', np.sqrt(mse_mnq_test4))
print('Root Mean Squarred Error mnq_test4C:', np.sqrt(mse_mnq_test4C))

### Advanced Slippage Simulation

In [ ]:

class trading_strategy_random_slip:
    def __init__(self):
        # --- Settings ---
        self.initial_account = 25000
        self.fee = 0.25     #Per part (no roundturn)
        self.point_value = 2
        
        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.account = []

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = round(min((self.account[-1] * 0.01) / (self.risk*self.point_value),(4 * self.account[-1]) / (self.opening_price*self.point_value)),0)
        
        self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, slip=0.25):
        remaining = units
        level = 0
        average_price = 0

        while remaining > 0:
            if level == 0:
                a = (5 - 10)/3 # Lim Inf Set  = 5
                initial_value = np.round(truncnorm(a, np.inf, loc = 10, scale = 3).rvs(),0)
                qty = min(initial_value, remaining)
            
            elif level == 1:
                value = round(np.round(uniform(loc = 0.6, scale = 0.3).rvs(),1) * initial_value,0)
                qty = min(value, remaining)
            
            elif level == 2:
                value = round(np.round(uniform(loc = 0.4, scale = 0.3).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            elif level >= 3:
                value = round(np.round(uniform(loc = 0.2, scale = 0.3).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            average_price += qty * exec_price
            remaining -= qty
            level += 1

        return average_price / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy')
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell')
            self.direction_history.append('Sell')


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = ((self.real_sell_price - self.real_buy_price) * self.units * self.point_value - 2 * self.units * self.fee)

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)


    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Buy": self.real_buy_price_history,
            "Average_Sell": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account}) 

In [ ]:
mnq_test4C_real = trading_strategy_random_slip()

for x in tqdm(range(0, len(mnq_test1_db))):

    if x == 0:
        mnq_test4C_real.starting_backtest(mnq_test1_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) > 0:
            mnq_test4C_real.buy_pos = 1
            mnq_test4C_real.buy_price = mnq_test1_db['open'].iloc[x]
            mnq_test4C_real.opening_price = mnq_test4C_real.buy_price
            mnq_test4C_real.stop_loss = min(mnq_test1_db['low'].iloc[x-1], mnq_test1_db['low'].iloc[x-2])
            mnq_test4C_real.take_profit = (mnq_test4C_real.opening_price - mnq_test4C_real.stop_loss) * 10 + mnq_test4C_real.opening_price
            mnq_test4C_real.risk = abs(mnq_test4C_real.opening_price - mnq_test4C_real.stop_loss)
            mnq_test4C_real.stop_to_be = False
            mnq_test4C_real.stop_step_1 = False

            mnq_test4C_real.apply_entry('Buy')
            real_risk = abs(mnq_test4C_real.real_buy_price - mnq_test4C_real.stop_loss)

        # SELL
        if (mnq_test1_db['close'].iloc[x-1] - mnq_test1_db['open'].iloc[x-2]) < 0:
            mnq_test4C_real.sell_pos = 1
            mnq_test4C_real.sell_price = mnq_test1_db['open'].iloc[x]
            mnq_test4C_real.opening_price = mnq_test4C_real.sell_price
            mnq_test4C_real.stop_loss = max(mnq_test1_db['high'].iloc[x-1], mnq_test1_db['high'].iloc[x-2])
            mnq_test4C_real.take_profit = (mnq_test4C_real.opening_price - mnq_test4C_real.stop_loss) * 10 + mnq_test4C_real.opening_price
            mnq_test4C_real.risk = abs(mnq_test4C_real.opening_price - mnq_test4C_real.stop_loss)
            mnq_test4C_real.stop_to_be = False
            mnq_test4C_real.stop_step_1 = False

            mnq_test4C_real.apply_entry('Sell')
            real_risk = abs(mnq_test4C_real.real_sell_price - mnq_test4C_real.stop_loss)


    # ================= LONG =================
    if mnq_test4C_real.buy_pos == 1 and mnq_test4C_real.sell_pos == 0:

        unrealized_profit = mnq_test1_db['close'].iloc[x] - mnq_test4C_real.real_buy_price

        # Stop loss Iniziale
        if mnq_test1_db['low'].iloc[x] <= mnq_test4C_real.stop_loss:
            mnq_test4C_real.sell_pos = 1
            mnq_test4C_real.sell_price = mnq_test4C_real.stop_loss
            mnq_test4C_real.apply_stop('Buy')

        # Take profit normale
        elif mnq_test1_db['high'].iloc[x] >= mnq_test4C_real.take_profit:
            mnq_test4C_real.sell_pos = 1
            mnq_test4C_real.sell_price = mnq_test4C_real.take_profit
            mnq_test4C_real.real_sell_price = mnq_test4C_real.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif (not mnq_test4C_real.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            mnq_test4C_real.stop_loss = mnq_test4C_real.real_buy_price + 5 * real_risk
            mnq_test4C_real.stop_step_1 = True
            mnq_test4C_real.stop_to_be = True

        elif (not mnq_test4C_real.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            mnq_test4C_real.stop_loss = mnq_test4C_real.real_buy_price  # stop a BE
            mnq_test4C_real.stop_to_be = True

    # ================= SHORT =================
    if mnq_test4C_real.buy_pos == 0 and mnq_test4C_real.sell_pos == 1:

        unrealized_profit = mnq_test4C_real.real_sell_price - mnq_test1_db['close'].iloc[x]

        # Stop loss iniziale
        if mnq_test1_db['high'].iloc[x] >= mnq_test4C_real.stop_loss:
            mnq_test4C_real.buy_pos = 1
            mnq_test4C_real.buy_price = mnq_test4C_real.stop_loss
            mnq_test4C_real.apply_stop('Sell')

        # Take profit normale
        elif mnq_test1_db['low'].iloc[x] <= mnq_test4C_real.take_profit:
            mnq_test4C_real.buy_pos = 1
            mnq_test4C_real.buy_price = mnq_test4C_real.take_profit
            mnq_test4C_real.real_buy_price = mnq_test4C_real.buy_price  # NO slippage

        # ================= STOP MANAGEMENT ================
        elif (not mnq_test4C_real.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            mnq_test4C_real.stop_loss = mnq_test4C_real.real_sell_price - 5 * real_risk
            mnq_test4C_real.stop_step_1 = True
            mnq_test4C_real.stop_to_be = True

        elif (not mnq_test4C_real.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            mnq_test4C_real.stop_loss = mnq_test4C_real.real_sell_price  # stop a BE
            mnq_test4C_real.stop_to_be = True

    # ================= END DAY =================
    if mnq_test1_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_test4C_real.buy_pos == 1 and mnq_test4C_real.sell_pos == 0:
            mnq_test4C_real.sell_pos = 1
            mnq_test4C_real.sell_price = mnq_test1_db['close'].iloc[x]
            mnq_test4C_real.real_sell_price = mnq_test1_db['close'].iloc[x]

        if mnq_test4C_real.buy_pos == 0 and mnq_test4C_real.sell_pos == 1:
            mnq_test4C_real.buy_pos = 1
            mnq_test4C_real.buy_price = mnq_test1_db['close'].iloc[x]
            mnq_test4C_real.real_buy_price = mnq_test1_db['close'].iloc[x]

    # ================= RECORD =================
    if mnq_test4C_real.buy_pos == 1 and mnq_test4C_real.sell_pos == 1:
        mnq_test4C_real.record_trade(mnq_test1_db['date'].iloc[x])
        mnq_test4C_real.reset()


### MNQ Test 4C Slip. VS MNQ Test 4C Adv. Slip.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = mnq_test4C.dates, y = mnq_test4C.account, mode = 'lines', name = 'MNQ Test 4C'))
fig.add_trace(go.Scatter(x = mnq_test4C_real.dates, y = mnq_test4C_real.account, mode = 'lines', name = 'MNQ Test 4C Adv. Slip'))
fig.update_layout(title = dict(text = "MNQ Test 4C Slip. VS MNQ Test 4C Adv. Slip.", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))
